In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Obsessive-Compulsive_Disorder"
cohort = "GSE78104"

# Input paths
in_trait_dir = "../../input/GEO/Obsessive-Compulsive_Disorder"
in_cohort_dir = "../../input/GEO/Obsessive-Compulsive_Disorder/GSE78104"

# Output paths
out_data_file = "../../output/preprocess/Obsessive-Compulsive_Disorder/GSE78104.csv"
out_gene_data_file = "../../output/preprocess/Obsessive-Compulsive_Disorder/gene_data/GSE78104.csv"
out_clinical_data_file = "../../output/preprocess/Obsessive-Compulsive_Disorder/clinical_data/GSE78104.csv"
json_path = "../../output/preprocess/Obsessive-Compulsive_Disorder/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Gene Expression Data Availability
# This dataset mentions mRNA expression data in the title and design,
# so it likely contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Identifying rows
trait_row = 1  # disease state
age_row = 3    # age
gender_row = 2 # gender

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert trait data to binary format (0 for control, 1 for OCD)"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'obsessive' in value.lower() or 'ocd' in value.lower():
        return 1
    elif 'normal' in value.lower() or 'control' in value.lower():
        return 0
    else:
        return None

def convert_age(value):
    """Convert age data to continuous format"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Extract numeric age from format like "25y"
    if 'y' in value:
        try:
            return float(value.replace('y', '').strip())
        except ValueError:
            return None
    
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value):
    """Convert gender data to binary format (0 for female, 1 for male)"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.lower() == 'male':
        return 1
    elif value.lower() == 'female':
        return 0
    else:
        return None

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the data
    preview_data = preview_df(selected_clinical_df)
    print("Preview of clinical data:")
    print(preview_data)
    
    # Save as CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [ ]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


### Step 4: Gene Identifier Review

In [ ]:
# Examining gene identifiers in data
# The identifiers look like microarray probe IDs rather than gene symbols
# The format is consistent with Agilent microarray designations (e.g., "A_19_P00315459")
# These are not standard human gene symbols and will need to be mapped

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# Check if there are any platforms defined in the SOFT file that might contain annotation data
with gzip.open(soft_file, 'rt') as f:
    soft_content = f.read()

# Look for platform sections in the SOFT file
platform_sections = re.findall(r'^!Platform_title\s*=\s*(.+)$', soft_content, re.MULTILINE)
if platform_sections:
    print(f"Platform title found: {platform_sections[0]}")

# Try to extract more annotation data by reading directly from the SOFT file
# Look for lines that might contain gene symbol mappings
symbol_pattern = re.compile(r'ID_REF\s+Symbol|ID\s+Gene Symbol', re.IGNORECASE)
annotation_lines = []
with gzip.open(soft_file, 'rt') as f:
    for line in f:
        if symbol_pattern.search(line):
            annotation_lines.append(line)
            # Collect the next few lines to see the annotation structure
            for _ in range(10):
                annotation_lines.append(next(f, ''))

if annotation_lines:
    print("Found potential gene symbol mappings:")
    for line in annotation_lines:
        print(line.strip())

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("\nGene annotation preview:")
print(preview_df(gene_annotation, n=10))

# If we need an alternative source of mapping, check if there are any other annotation files in the cohort directory
cohort_files = os.listdir(in_cohort_dir)
annotation_files = [f for f in cohort_files if 'annotation' in f.lower() or 'platform' in f.lower()]
if annotation_files:
    print("\nAdditional annotation files found in the cohort directory:")
    for file in annotation_files:
        print(file)


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Identify which columns in the gene annotation contain probe IDs and gene symbols
# From the preview, 'ID' contains probe identifiers that match expression data
# 'GeneSymbol' appears to contain gene symbols, but we should check for nulls
probe_col = 'ID'
gene_col = 'GeneSymbol'

print(f"Mapping probes using columns: {probe_col} → {gene_col}")
print(f"Number of probes in annotation: {len(gene_annotation)}")
print(f"Number of non-null gene symbols: {gene_annotation[gene_col].notna().sum()}")

# 2. Get the gene mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, prob_col=probe_col, gene_col=gene_col)
print(f"Mapping dataframe shape: {mapping_df.shape}")
print("First few rows of mapping:")
print(mapping_df.head())

# 3. Apply gene mapping to convert probe-level measurements to gene-level expression
# This will handle the many-to-many relationship properly
gene_data = apply_gene_mapping(expression_df=gene_data, mapping_df=mapping_df)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First few gene symbols:")
print(gene_data.index[:10])

# Save gene expression data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


### Step 7: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the gene expression data
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data.shape}")
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load clinical data that was already saved in Step 2
clinical_data_df = pd.read_csv(out_clinical_data_file)
# Set proper index for clinical data 
clinical_data_df.set_index(clinical_data_df.columns[0], inplace=True)
print(f"Loaded clinical data shape: {clinical_data_df.shape}")
print(clinical_data_df.head())

# Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_data_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
print(linked_data.iloc[:5, :5] if not linked_data.empty else "Linked data is empty")

# 3. Handle missing values
# Identify trait column - it's the first column in the linked data
trait_col = linked_data.columns[0]  # This gets the actual column name where trait values are stored
linked_data = handle_missing_values(linked_data, trait_col)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 4. Check for bias in features
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait_col)

# 5. Validate and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from peripheral blood of patients with Obsessive-Compulsive Disorder."
)

# 6. Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for analysis. No linked data file saved.")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the gene expression data - we already did this in step 6
# No need to do it again
print(f"Gene data already normalized and saved to {out_gene_data_file}")

# 2. Load clinical data that was already saved in Step 2
clinical_data_df = pd.read_csv(out_clinical_data_file, index_col=0)
print(f"Loaded clinical data shape: {clinical_data_df.shape}")
print(clinical_data_df.head())

# 2. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_data_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
print(linked_data.iloc[:5, :5] if not linked_data.empty else "Linked data is empty")

# 3. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 4. Check for bias in features
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Validate and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from dorsolateral prefrontal cortex of postmortem tissue with Eating Disorders."
)

# 6. Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for analysis. No linked data file saved.")